### Generador de musica usando Keras sobre TensorFlow y backend

By Elías Jesús Ventura-Molina

Proceso:
1. Leer un archivo midi, converirlo a una matriz de caracteristicas
2. Crear un modelo de LSTM con Keras para aprender los patrones
3. Usar una muestra del archivo midi para crear otra melodia
4. Guardar la prediccion como un archivo midi


### Librerias Necesarias

Mido para trabajar con archivos midi.    
Las capas que se necesitaran son:      
1. LSTM
2. Dense
3. Activation
4. Dropout
5. Flatten

El modelo de la red será secuencial

In [1]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np

Using TensorFlow backend.


##### Leer el archivo midi

Canción de Pokemon

In [2]:
mid = MidiFile('Samples/Nintendo_-_Pokemon_Fire_Red_Route_1_Piano_Cover_Hard_Version.mid') 

##### Extraer las notas de la secuencia

Se le llama secuencia a la cancion, ya que es una secuencia de notas a traves del tiempo.
Los archivos midi estan compuestos por pisas, cada pista contiene una lista de mensajes y meta mensajes con el tiempo como atributo.

In [3]:
notes = []
for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        data = msg.bytes()
        notes.append(data[1])

#### Aplicar min-max scalling

Sirve para normalizar los datos 

In [4]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))

#### Preparar los datos para entrenar y sacar una muestra para realizar predicciones

In [5]:
# Las capas LSTM necesitan que los datos tengan un tamaño especifico
# crear una lista de listas
notes = [list(note) for note in notes]

# muestras para entrenamiento y predicción
X = []
y = []
# numero de notas por lote
n_prev = 30
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])
# guardar una semilla para realizar predicciones
X_test = X[-300:]
X = X[:-300]
y = y[:-300]

#### Modelo

Añadir las capas necesarias al modelo, las capas LSTM son dependientes del tiempo      
Guardar checkpoints      
Documentacion de capas LSTM: https://keras.io/api/layers/recurrent_layers/lstm/

In [11]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('linear'))
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)

# Carpeta donde se guardara el checkpoint cada 5 epocas

filepath="./Checkpoints/checkpoint_model_{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(filepath, monitor='val_acc', 
                                      verbose=1, save_best_only=False, 
                                      mode='auto', period=5)

#### Entrenamiento


In [12]:
model.fit(np.array(X), np.array(y), 32, 10, verbose=1, callbacks=[model_save_callback])

Epoch 1/10
12770/12770 [==============================] - 39s 3ms/step - loss: 0.0504
Epoch 2/10
12770/12770 [==============================] - 38s 3ms/step - loss: 0.0463
Epoch 3/10
12770/12770 [==============================] - 39s 3ms/step - loss: 0.0448
Epoch 4/10
12770/12770 [==============================] - 38s 3ms/step - loss: 0.0420
Epoch 5/10
12770/12770 [==============================] - 39s 3ms/step - loss: 0.0317

Epoch 00005: saving model to ./Checkpoints/checkpoint_model_05.hdf5
Epoch 6/10
12770/12770 [==============================] - 38s 3ms/step - loss: 0.0257
Epoch 7/10
12770/12770 [==============================] - 39s 3ms/step - loss: 0.0220
Epoch 8/10
12770/12770 [==============================] - 39s 3ms/step - loss: 0.0195
Epoch 9/10
12770/12770 [==============================] - 40s 3ms/step - loss: 0.0166
Epoch 10/10
12770/12770 [==============================] - 38s 3ms/step - loss: 0.0120

Epoch 00010: saving model to ./Checkpoints/checkpoint_model_10.hdf5


#### Realizar predicciones

In [13]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

#### Guardar la prediccion como un archivo midi

In [14]:
mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 significa note_on
    # 50 es la velocidad
    note = np.asarray([147, note, 50])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('LSTM_music.mid')